In [52]:
#!pip install keras
#!pip install tensorflow
#!pip install torch==1.4.0
#!pip install sentencepiece
!pip install --upgrade transformers==3.0.2 # the authors probably used version 3.0.2
# !pip install contractions
# !pip install unidecode
# !pip install contractions
# nltk.download('punkt')

ERROR: Invalid requirement: '#'


In [53]:
# the basics
import pandas as pd
import numpy as np
import io
import os
import logging
# import random

# data cleaning
# import re
# import contractions as ct
# import string
# import unidecode 

# math + machine learning
from scipy.stats import spearmanr
from math import floor, ceil
from tqdm import tqdm # for nice progress meters
import sklearn
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import nltk 
import torch
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras 

# import keras model and layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Concatenate
# import tensorflow.keras.utils.Sequence
# from transformers import *
import transformers
from transformers import TFBertModel, BertTokenizer
from transformers import BertTokenizer, TFBertModel
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# viz
import seaborn as sns
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)
print(tf.__version__)
print(transformers.__version__)

2.8.0
3.0.2


# Baseline Model
- 1-layer NN that predicts binary humor label using pre-trained BERT embeddings of 10K short jokes/non-jokes
- inputs: input IDs, attention masks, and token type IDs from pre-trained BERT tokenizer
- outputs: probability of a short text being humorous (p > 0.5 = humorous)

In [91]:
### basic model architecture ###

INPUT_LEN = 100 # same as MAX_LENGTH in bertembeddings.py

x1 = Input(shape=(INPUT_LEN,), dtype=tf.int32, name='input_ids')
x2 = Input(shape=(INPUT_LEN,), dtype=tf.int32, name='attention_masks')
x3 = Input(shape=(INPUT_LEN,), dtype=tf.int32, name='token_type_ids')

bert_model = TFBertModel.from_pretrained('bert-base-uncased')
bert_embeddings = bert_model(x1, attention_mask=x2, token_type_ids=x3)
pooled_embeddings = bert_embeddings[1]

yhat = Dense(1, activation='sigmoid', name='output')(pooled_embeddings)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [92]:
### intialize and compile model ###
base_model = Model([x1, x2, x3], [yhat], name='baseline')
base_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = ['accuracy'])

### train model ###
# load data
inputs = np.load("testrun_model_train_inputs.npz", allow_pickle=True)
X = list(inputs.values())[-3:]
data = pd.read_csv('clean_dataset.csv')
y_train, y_test = data['humor'][:160000][:50], data['humor'][160000:][:50]

# fit model
base_hist = base_model.fit(X, y_train.values, epochs=5, batch_size=6)

Epoch 1/5
9/9 [==============================] - 83s 4s/step - loss: 1.0951 - accuracy: 0.5400
Epoch 2/5
9/9 [==============================] - 34s 4s/step - loss: 0.7284 - accuracy: 0.4600
Epoch 3/5
9/9 [==============================] - 34s 4s/step - loss: 0.7281 - accuracy: 0.5400
Epoch 4/5
9/9 [==============================] - 39s 4s/step - loss: 0.8475 - accuracy: 0.4800
Epoch 5/5
9/9 [==============================] - 38s 4s/step - loss: 0.7856 - accuracy: 0.4200


In [109]:
# the baseline model does worse than simply guessing the dominant class
dom_acc = y_train.mean() if y_train.mean() > 0.5 else 1 - y_train.mean()
print("Guessing the dominant class gives an accuracy of {}".format(round(dom_acc,2)))

Guessing the dominant class gives an accuracy of 0.58


In [107]:
# model always guesses the dominant class
(base_model.predict(X) > 0.5).flatten()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [103]:
### predict test data and print test accuracy ###
test_inputs = np.load("testrun_model_test_inputs.npz", allow_pickle=True)
test_X = list(test_inputs.values())[-3:]
y_pred = (base_model.predict(test_X) > 0.5).flatten()
print("Accuracy = {}".format(round(accuracy_score(y_test, y_pred)*100, 2)))
print("F1 score = {}".format((f1_score(y_test, y_pred))))

Accuracy = 52.0
F1 score = 0.0


In [105]:
# model always classifies document as non-joke
# (non-joke is the dominant class in the 50 training examples)
y_pred

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])